<a href="https://colab.research.google.com/github/Mamitakp/VN2_B03_127/blob/main/ALLtask3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install spark 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 79 kB/s 
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58762 sha256=ad43dd243f2f13f0baaf37abe53d25ef79ea8bc4bd01aa7c0d23940fc2ee5e04
  Stored in directory: /root/.cache/pip/wheels/4e/0e/f1/164619f9920fb447d294afaae11a7715bd442ded7225953d72
Successfully built spark


In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 65.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=ff38ba106bdc0f5aad3519d283e40ac452e50b922625069c093728c36916e315
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
from pyspark.sql import *
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('colab').getOrCreate()

In [4]:
df=spark.read.csv(r'/content/data_prep .csv',inferSchema=True,header=True)

In [5]:
df.show()

+------+----------------+----------+------+-----------+
|userid|         emailid|      date|amount|   discount|
+------+----------------+----------+------+-----------+
|     1| test1@gmail.com|01-01-2019|   100|        1.0|
|     2| test2@gmail.com|02-01-2019|   100|        1.0|
|     3| test3@gmail.com|03-01-2019|   101|        2.0|
|     4| test4@gmail.com|04-01-2019|   102|       10.0|
|     5| test5@gmail.com|05-01-2019| 102.5|        2.0|
|     6| test6@gmail.com|06-01-2019| 103.2|        1.0|
|     7| test7@gmail.com|07-01-2019| 103.9|        3.0|
|     8| test8@gmail.com|08-01-2019| 104.6|        5.0|
|     9| test9@gmail.com|09-01-2019| 105.3|4.571428571|
|    10|test10@gmail.com|10-01-2019|   106|4.892857143|
|    11|test11@gmail.com|11-01-2019| 106.7|5.214285714|
|    12|test12@gmail.com|12-01-2019| 107.4|5.535714286|
|    13|test13@gmail.com|13-01-2019| 108.1|5.857142857|
|    14|test14@gmail.com|14-01-2019| 108.8|6.178571429|
|    15|test15@gmail.com|15-01-2019| 109.5|     

In [37]:
from datetime import datetime

def try_parsing_date(text):
    for fmt in ('%Y-%m-%d','%dd-%mm-%yyyy', '%d.%m.%Y','%d-%m-%Y'):
        try:
            date_object=datetime.strptime(text, fmt)
            converted_date_object=date_object.strftime('%Y-%m-%d')
            return converted_date_object

        except ValueError:
            pass
    raise ValueError('no valid date format found')

In [38]:
try_parsing_date('18-01-2019')

'2019-01-18'

In [39]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


date_udf= udf(lambda val:try_parsing_date(val))

In [41]:
result=df.select(col('date'),date_udf(col('date')).alias('date_t')).show()

+----------+----------+
|      date|    date_t|
+----------+----------+
|01-01-2019|2019-01-01|
|02-01-2019|2019-01-02|
|03-01-2019|2019-01-03|
|04-01-2019|2019-01-04|
|05-01-2019|2019-01-05|
|06-01-2019|2019-01-06|
|07-01-2019|2019-01-07|
|08-01-2019|2019-01-08|
|09-01-2019|2019-01-09|
|10-01-2019|2019-01-10|
|11-01-2019|2019-01-11|
|12-01-2019|2019-01-12|
|13-01-2019|2019-01-13|
|14-01-2019|2019-01-14|
|15-01-2019|2019-01-15|
|16-01-2019|2019-01-16|
|17-01-2019|2019-01-17|
|18-01-2019|2019-01-18|
|19-01-2019|2019-01-19|
|20-01-2019|2019-01-20|
+----------+----------+
only showing top 20 rows



In [44]:
df.count()

902

In [43]:
df.dropna().count()

901

In [45]:
df.dropDuplicates().count()

901

In [46]:
pip install validate_email

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for validate-email: filename=validate_email-1.3-py3-none-any.whl size=5482 sha256=eace03a2697ce5a1471340daf77f9d2a156886794ea447f14611ceef84abab3d
  Stored in directory: /root/.cache/pip/wheels/ff/8f/92/c43287715852eaa75e0d8aa1941c481072b4a82c4f4975074e
Successfully built validate-email


In [53]:
from validate_email import validate_email


In [54]:
is_valid=validate_email('test13@gmail.com')

In [63]:
def valid_parse(str1):
  is_valid=validate_email(str1)
  if is_valid:
    return str1
  else:
    'error'


In [64]:
print(is_valid)

True


In [65]:
email_udf=udf(lambda x:valid_parse(x))

In [66]:
final_result=df.select(col('emailid'),email_udf(col('emailid')).alias('new_res'))

In [67]:
final_result.show(n=10)

+----------------+----------------+
|         emailid|         new_res|
+----------------+----------------+
| test1@gmail.com| test1@gmail.com|
| test2@gmail.com| test2@gmail.com|
| test3@gmail.com| test3@gmail.com|
| test4@gmail.com| test4@gmail.com|
| test5@gmail.com| test5@gmail.com|
| test6@gmail.com| test6@gmail.com|
| test7@gmail.com| test7@gmail.com|
| test8@gmail.com| test8@gmail.com|
| test9@gmail.com| test9@gmail.com|
|test10@gmail.com|test10@gmail.com|
+----------------+----------------+
only showing top 10 rows

